In [89]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os 
from datetime import datetime
import pandas as pd 
import numpy as np 
import keras
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import TimeSeriesSplit 
from sklearn.model_selection import train_test_split
from keras.models import Sequential 
from keras.layers import Dense, Flatten, Dropout, LSTM, GRU
from sklearn.metrics import mean_squared_error
from keras.layers import Dropout,BatchNormalization

os.chdir(r'D:\Github 작업용\Github_cowork_practice')

In [90]:
train_data = pd.read_csv('Embedding_train_data.csv')
test_data = pd.read_csv('Embedding_test_data.csv')

In [91]:
train_data

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,2016-02-01,월,2601,50,150,238,0.0,"['모닝롤', '찐빵', '우유', '두유', '주스', '계란후라이', '호두죽'...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '오징어찌개', '쇠불고기',...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '육개장', '자반고등어구이'...",...,-0.677969,0.026655,0.521240,0.106772,-0.151797,0.285805,-0.916271,-0.327705,-0.489872,-0.114326
1,2016-02-02,화,2601,50,173,319,0.0,"['모닝롤', '단호박샌드', '우유', '두유', '주스', '계란후라이', '팥...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '김치찌개', '가자미튀김',...","['콩나물밥*양념장', '(쌀,현미흑미:국내산)', '어묵국', '유산슬', '(쇠...",...,0.218089,-0.672441,0.787501,0.266674,-0.693809,0.412709,-0.846675,-0.540325,-0.346797,0.396869
2,2016-02-03,수,2601,56,180,111,0.0,"['모닝롤', '베이글', '우유', '두유', '주스', '계란후라이', '표고버...","['카레덮밥', '(쌀,현미흑미:국내산)', '팽이장국', '치킨핑거', '(닭고기...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '청국장찌개', '황태양념구이...",...,-0.149827,0.259681,0.897299,0.419382,0.033570,0.816938,-0.607790,-0.501021,0.063332,0.382068
3,2016-02-04,목,2601,104,220,355,0.0,"['모닝롤', '토마토샌드', '우유', '두유', '주스', '계란후라이', '닭...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '쇠고기무국', '주꾸미볶음'...","['미니김밥*겨자장', '(쌀,현미흑미:국내산)', '우동', '멕시칸샐러드', '...",...,-0.257097,-0.550777,1.180077,0.568659,-0.483393,1.447895,-0.850024,-0.630203,-0.074369,0.335166
4,2016-02-05,금,2601,278,181,34,0.0,"['모닝롤', '와플', '우유', '두유', '주스', '계란후라이', '쇠고기죽...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '떡국', '돈육씨앗강정', ...","['쌀밥', '잡곡밥', '(쌀,현미흑미:국내산)', '차돌박이찌개', '(쇠고기:...",...,-0.289660,0.011186,1.035358,0.692184,-0.199249,0.299185,-0.974069,-0.377276,-0.117614,-0.063785
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,수,2983,75,198,4,391.0,"['모닝롤', '페퍼로니피자', '우유', '주스', '계란후라이', '찐계란', ...","['쌀밥', '흑미밥', '찰현미밥', '아욱국', '수제함박스테이크', '견과류마...","['김치볶음밥', '미니쫄우동*맛살튀김', '브로콜리깨소스무침', '계란후라이', ...",...,0.157136,-0.048317,0.516620,-0.263557,0.738902,0.129731,0.395461,-0.805688,-0.497606,0.216988
1201,2021-01-21,목,2983,92,231,462,351.0,"['모닝롤', '생크림단팥빵', '우유', '주스', '계란후라이', '찐계란', ...","['쌀밥', '수수밥', '찰현미밥', '냉이된장국', '동파육', '봄동전', '...","['흑미밥', '쇠고기무국', '삼치양념구이', '비엔나채소볶음', '숙주나물당근무...",...,0.681265,-0.339047,0.113666,0.122566,0.351240,-0.328923,-0.620603,-0.696990,0.313707,0.084509
1202,2021-01-22,금,2983,255,248,1,303.0,"['모닝롤', 'BLT샌드위치', '우유', '주스', '계란후라이', '찐계란',...","['전주비빔밥*약고추장', '계란파국', '요거닭', '올방개묵무침', '파프리카해...","['흑미밥', '수제비국', '수제맛쵸킹탕수육', '유부채소겨자냉채', '참나물무침...",...,1.046267,-0.648388,0.122432,0.036668,0.654255,-0.277724,-0.561344,-0.290120,-0.088212,-0.819337
1203,2021-01-25,월,2983,107,153,616,327.0,"['모닝롤', '호박고구마오븐구이', '우유', '주스', '계란후라이', '찐계란...","['쌀밥', '흑미밥', '찰현미밥', '전주식콩나물해장국', '돈육간장불고기', ...","['흑미밥', '열무된장국', '장어강정*데리야끼소스', '깻잎쌈*생강채', '오이...",...,1.090084,0.055053,0.210113,-0.062785,-0.246567,-0.333712,0.029828,-0.845187,-0.185106,-0.561755


In [92]:
test_data

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,...,석식41,석식42,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50
0,2021-01-27,수,2983,88,182,5,358.0,"['모닝롤', '연유버터베이글', '우유', '주스', '계란후라이', '찐계란',...","['쌀밥', '흑미밥', '찰현미밥', '대구지리', '매운돈갈비찜', '오꼬노미계...","['흑미밥', '얼큰순두부찌개', '쇠고기우엉볶음', '버섯햄볶음', '(New)아...",...,0.855815,-0.676641,0.008660,0.049113,0.279932,-0.255711,-0.784561,-0.287553,-0.123809,-0.417225
1,2021-01-28,목,2983,104,212,409,348.0,"['모닝롤', '대만샌드위치', '우유', '주스', '계란후라이', '찐계란', ...","['쌀밥', '보리밥', '찰현미밥', '우렁된장찌개', '오리주물럭', '청양부추...","['충무김밥', '우동국물', '오징어무침', '꽃맛살샐러드', '얼갈이쌈장무침',...",...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2021-01-29,금,2983,270,249,0,294.0,"['모닝롤', '핫케익', '우유', '주스', '계란후라이', '찐계란', '오곡...","['쌀밥', '흑미밥', '찰현미밥', '팽이장국', '수제돈까스*소스', '가자미...","['흑미밥', '물만둣국', '카레찜닭', '숯불양념꼬지어묵', '꼬시래기무침', ...",...,1.274020,-0.963867,0.068798,0.263769,0.199401,-0.475091,-1.022334,-0.346184,0.053542,-0.519112
3,2021-02-01,월,2924,108,154,538,322.0,"['모닝롤', '촉촉한치즈케익', '우유', '주스', '계란후라이', '찐계란',...","['쌀밥', '흑미밥', '찰현미밥', '배추들깨국', '오리대패불고기', '시금치...","['흑미밥', '동태탕', '돈육꽈리고추장조림', '당면채소무침', '모자반무침',...",...,1.215914,-0.308417,0.200664,-0.110430,0.222746,-0.128564,-0.394535,-0.758083,-0.181848,-0.035184
4,2021-02-02,화,2924,62,186,455,314.0,"['모닝롤', '토마토샌드', '우유', '주스', '계란후라이', '찐계란', '...","['쌀밥', '팥밥', '찰현미밥', '부대찌개', '닭살데리야끼조림', '버섯탕수...","['흑미밥', '바지락살국', '쇠고기청경채볶음', '두부구이*볶은김치', '머위된...",...,2.021988,-0.430297,-0.483459,-0.619689,0.896096,-0.793497,-0.771993,-0.141780,0.286809,-0.650598
5,2021-02-03,수,2924,59,199,5,286.0,"['모닝롤', '게살모닝샌드', '우유', '주스', '계란후라이', '찐계란', ...","['쌀밥', '흑미밥', '찰현미밥', '아욱국', '매콤해물볶음', '감자조림',...","['오므라이스', '가쓰오장국', '빌소세지구이*구운채소', '단감치커리무침', '...",...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,2021-02-04,목,2924,61,211,476,288.0,"['모닝롤', '사과파이', '우유', '주스', '계란후라이', '찐계란', '누...","['쌀밥', '차조밥', '찰현미밥', '설렁탕', '고등어김치말이찜', '볼어묵굴...","['흑미밥', '계란파국', '돈육두루치기', '감자채파프리카볶음', '세발나물오리...",...,1.274020,-0.963867,0.068798,0.263769,0.199401,-0.475091,-1.022334,-0.346184,0.053542,-0.519112
7,2021-02-05,금,2924,169,252,0,256.0,"['모닝롤', '앙버터모닝빵', '우유', '주스', '계란후라이', '찐계란', ...","['쌀밥', '흑미밥', '찰현미밥', '북엇국', '닭볶음탕', '채소전*장', ...","['유부초밥', '추가밥', '온메밀소바', '국물떡볶이', '순대찜*소금', '청...",...,-0.805625,-0.938924,1.104038,1.250359,-1.304088,1.062914,-0.232308,-1.033581,-0.420452,0.128777
8,2021-02-08,월,2924,88,174,690,329.0,"['모닝롤', '스콘', '우유', '주스', '계란후라이', '찐계란', '누룽지...","['쌀밥', '흑미밥', '찰현미밥', '감자양파국', '돈수육*씨앗쌈장', '매콤...","['흑미밥', '냉이국', '반반치킨', '꼬막채소무침', '청경채찜', '포기김치']",...,1.137758,-0.795042,-0.075241,0.048106,0.274380,-0.382176,-0.622488,-0.425739,0.116015,-0.470904
9,2021-02-09,화,2924,94,183,542,329.0,"['모닝롤', '치즈팡샌드', '우유', '주스', '계란후라이', '찐계란', '...","['쌀밥', '기장밥', '찰현미밥', '장각백숙', '적어양념장구이', '채소스틱...","['흑미밥', '미역국', '매운소불고기', '단호박두부탕수', '메추리알장조림',...",...,1.894431,-0.407195,-0.303128,-1.115092,0.632459,-1.117839,-0.496161,-0.377495,0.645056,-0.565803


# 전처리 실시한다

일자의 '월', '일' 만 사용할 예정이며 조식/중식/석식 메뉴는 제거한다.

In [93]:
train_data = train_data.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis = 1)
test_data = test_data.drop(['조식메뉴', '중식메뉴', '석식메뉴'], axis = 1)

In [94]:
train_data['월'] = train_data.일자.str[5:7]
train_data['일'] = train_data.일자.str[8:10]

test_data['월'] = train_data.일자.str[5:7]
test_data['일'] = train_data.일자.str[8:10]

In [95]:
train_data

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,...,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50,월,일
0,2016-02-01,월,2601,50,150,238,0.0,1039.0,331.0,0.345063,...,0.521240,0.106772,-0.151797,0.285805,-0.916271,-0.327705,-0.489872,-0.114326,02,01
1,2016-02-02,화,2601,50,173,319,0.0,867.0,560.0,0.537751,...,0.787501,0.266674,-0.693809,0.412709,-0.846675,-0.540325,-0.346797,0.396869,02,02
2,2016-02-03,수,2601,56,180,111,0.0,1017.0,573.0,0.430975,...,0.897299,0.419382,0.033570,0.816938,-0.607790,-0.501021,0.063332,0.382068,02,03
3,2016-02-04,목,2601,104,220,355,0.0,978.0,525.0,0.695163,...,1.180077,0.568659,-0.483393,1.447895,-0.850024,-0.630203,-0.074369,0.335166,02,04
4,2016-02-05,금,2601,278,181,34,0.0,925.0,330.0,0.519855,...,1.035358,0.692184,-0.199249,0.299185,-0.974069,-0.377276,-0.117614,-0.063785,02,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,2021-01-20,수,2983,75,198,4,391.0,1093.0,421.0,0.205926,...,0.516620,-0.263557,0.738902,0.129731,0.395461,-0.805688,-0.497606,0.216988,01,20
1201,2021-01-21,목,2983,92,231,462,351.0,832.0,353.0,-0.045411,...,0.113666,0.122566,0.351240,-0.328923,-0.620603,-0.696990,0.313707,0.084509,01,21
1202,2021-01-22,금,2983,255,248,1,303.0,579.0,217.0,0.246507,...,0.122432,0.036668,0.654255,-0.277724,-0.561344,-0.290120,-0.088212,-0.819337,01,22
1203,2021-01-25,월,2983,107,153,616,327.0,1145.0,502.0,0.092769,...,0.210113,-0.062785,-0.246567,-0.333712,0.029828,-0.845187,-0.185106,-0.561755,01,25


In [96]:
test_data

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식1,조식2,조식3,...,석식43,석식44,석식45,석식46,석식47,석식48,석식49,석식50,월,일
0,2021-01-27,수,2983,88,182,5,358.0,0.318706,1.173837,0.228873,...,0.008660,0.049113,0.279932,-0.255711,-0.784561,-0.287553,-0.123809,-0.417225,02,01
1,2021-01-28,목,2983,104,212,409,348.0,0.273322,0.871655,0.077482,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,02,02
2,2021-01-29,금,2983,270,249,0,294.0,0.068761,1.216680,0.287047,...,0.068798,0.263769,0.199401,-0.475091,-1.022334,-0.346184,0.053542,-0.519112,02,03
3,2021-02-01,월,2924,108,154,538,322.0,0.051651,1.120480,0.057369,...,0.200664,-0.110430,0.222746,-0.128564,-0.394535,-0.758083,-0.181848,-0.035184,02,04
4,2021-02-02,화,2924,62,186,455,314.0,0.405712,1.120068,0.380671,...,-0.483459,-0.619689,0.896096,-0.793497,-0.771993,-0.141780,0.286809,-0.650598,02,05
5,2021-02-03,수,2924,59,199,5,286.0,0.178310,1.330413,0.157849,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,02,11
6,2021-02-04,목,2924,61,211,476,288.0,0.078337,0.896581,0.389845,...,0.068798,0.263769,0.199401,-0.475091,-1.022334,-0.346184,0.053542,-0.519112,02,12
7,2021-02-05,금,2924,169,252,0,256.0,0.060798,1.234879,0.586699,...,1.104038,1.250359,-1.304088,1.062914,-0.232308,-1.033581,-0.420452,0.128777,02,15
8,2021-02-08,월,2924,88,174,690,329.0,0.089839,0.911408,0.390926,...,-0.075241,0.048106,0.274380,-0.382176,-0.622488,-0.425739,0.116015,-0.470904,02,16
9,2021-02-09,화,2924,94,183,542,329.0,0.124511,1.134649,0.235986,...,-0.303128,-1.115092,0.632459,-1.117839,-0.496161,-0.377495,0.645056,-0.565803,02,17


일자는 이제 안쓸거니까 갖다 버린다.

In [97]:
train_data = train_data.drop(['일자'], axis = 1)
test_data = test_data.drop(['일자'], axis = 1)

범주형은 단 하나, 요일 변수가 존재한다.

In [98]:
from sklearn.preprocessing import OneHotEncoder

o_scaler = OneHotEncoder(sparse=False)

train_lab = o_scaler.fit_transform(train_data[['요일']])
test_lab = o_scaler.transform(test_data[['요일']])

train_lab = pd.DataFrame(train_lab, columns = ['요일1','요일2','요일3','요일4','요일5'])
test_lab = pd.DataFrame(test_lab, columns = ['요일1','요일2','요일3','요일4','요일5'])

train_data = pd.concat([train_data, train_lab], axis = 1)
test_data = pd.concat([test_data, test_lab], axis = 1)

In [99]:
train_data

,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,조식2,...,석식48,석식49,석식50,월,일,요일1,요일2,요일3,요일4,요일5
0,월,2601,50,150,238,0.0,1039.0,331.0,0.345063,0.841029,...,-0.327705,-0.489872,-0.114326,02,01,0.0,0.0,0.0,1.0,0.0
1,화,2601,50,173,319,0.0,867.0,560.0,0.537751,0.772565,...,-0.540325,-0.346797,0.396869,02,02,0.0,0.0,0.0,0.0,1.0
2,수,2601,56,180,111,0.0,1017.0,573.0,0.430975,0.564405,...,-0.501021,0.063332,0.382068,02,03,0.0,0.0,1.0,0.0,0.0
3,목,2601,104,220,355,0.0,978.0,525.0,0.695163,0.915111,...,-0.630203,-0.074369,0.335166,02,04,0.0,1.0,0.0,0.0,0.0
4,금,2601,278,181,34,0.0,925.0,330.0,0.519855,0.525534,...,-0.377276,-0.117614,-0.063785,02,05,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,수,2983,75,198,4,391.0,1093.0,421.0,0.205926,1.241737,...,-0.805688,-0.497606,0.216988,01,20,0.0,0.0,1.0,0.0,0.0
1201,목,2983,92,231,462,351.0,832.0,353.0,-0.045411,1.020490,...,-0.696990,0.313707,0.084509,01,21,0.0,1.0,0.0,0.0,0.0
1202,금,2983,255,248,1,303.0,579.0,217.0,0.246507,1.052156,...,-0.290120,-0.088212,-0.819337,01,22,1.0,0.0,0.0,0.0,0.0
1203,월,2983,107,153,616,327.0,1145.0,502.0,0.092769,1.107356,...,-0.845187,-0.185106,-0.561755,01,25,0.0,0.0,0.0,1.0,0.0


In [100]:
test_data

,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식1,조식2,조식3,조식4,...,석식48,석식49,석식50,월,일,요일1,요일2,요일3,요일4,요일5
0,수,2983,88,182,5,358.0,0.318706,1.173837,0.228873,-0.016794,...,-0.287553,-0.123809,-0.417225,02,01,0.0,0.0,1.0,0.0,0.0
1,목,2983,104,212,409,348.0,0.273322,0.871655,0.077482,0.143496,...,0.000000,0.000000,0.000000,02,02,0.0,1.0,0.0,0.0,0.0
2,금,2983,270,249,0,294.0,0.068761,1.216680,0.287047,0.108864,...,-0.346184,0.053542,-0.519112,02,03,1.0,0.0,0.0,0.0,0.0
3,월,2924,108,154,538,322.0,0.051651,1.120480,0.057369,0.071547,...,-0.758083,-0.181848,-0.035184,02,04,0.0,0.0,0.0,1.0,0.0
4,화,2924,62,186,455,314.0,0.405712,1.120068,0.380671,0.311570,...,-0.141780,0.286809,-0.650598,02,05,0.0,0.0,0.0,0.0,1.0
5,수,2924,59,199,5,286.0,0.178310,1.330413,0.157849,0.013180,...,0.000000,0.000000,0.000000,02,11,0.0,0.0,1.0,0.0,0.0
6,목,2924,61,211,476,288.0,0.078337,0.896581,0.389845,0.257165,...,-0.346184,0.053542,-0.519112,02,12,0.0,1.0,0.0,0.0,0.0
7,금,2924,169,252,0,256.0,0.060798,1.234879,0.586699,0.087984,...,-1.033581,-0.420452,0.128777,02,15,1.0,0.0,0.0,0.0,0.0
8,월,2924,88,174,690,329.0,0.089839,0.911408,0.390926,-0.014367,...,-0.425739,0.116015,-0.470904,02,16,0.0,0.0,0.0,1.0,0.0
9,화,2924,94,183,542,329.0,0.124511,1.134649,0.235986,-0.142773,...,-0.377495,0.645056,-0.565803,02,17,0.0,0.0,0.0,0.0,1.0


In [101]:
train_data = train_data.drop(['요일'], axis = 1)
test_data = test_data.drop(['요일'], axis = 1)

최종 input 형태는 다음과 같다.

In [102]:
train_data.head()

,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,중식계,석식계,조식1,조식2,조식3,...,석식48,석식49,석식50,월,일,요일1,요일2,요일3,요일4,요일5
0,2601,50,150,238,0.0,1039.0,331.0,0.345063,0.841029,-0.244781,...,-0.327705,-0.489872,-0.114326,02,01,0.0,0.0,0.0,1.0,0.0
1,2601,50,173,319,0.0,867.0,560.0,0.537751,0.772565,-0.227728,...,-0.540325,-0.346797,0.396869,02,02,0.0,0.0,0.0,0.0,1.0
2,2601,56,180,111,0.0,1017.0,573.0,0.430975,0.564405,0.014996,...,-0.501021,0.063332,0.382068,02,03,0.0,0.0,1.0,0.0,0.0
3,2601,104,220,355,0.0,978.0,525.0,0.695163,0.915111,-0.464399,...,-0.630203,-0.074369,0.335166,02,04,0.0,1.0,0.0,0.0,0.0
4,2601,278,181,34,0.0,925.0,330.0,0.519855,0.525534,-0.039259,...,-0.377276,-0.117614,-0.063785,02,05,1.0,0.0,0.0,0.0,0.0


이제 train / train_target / test의 분할을 실시한다.

In [103]:
train_target = train_data.loc[:,['중식계','석식계']]
train_data = train_data.drop(['중식계','석식계'],axis=1)

In [104]:
#정규화
# onehot부분이 중간에 중복으로 들어가긴 했지만, 어짜피 값 변화가 없으므로 그냥 그대로 진행한다.
data_scaler = MinMaxScaler()
data_scaled = data_scaler.fit_transform(train_data)

target_scaler = MinMaxScaler()
target_scaled = target_scaler.fit_transform(train_target)


In [105]:
# train set, test set 분할 
x_train, x_valid, y_train, y_valid = train_test_split(data_scaled, target_scaled, test_size=0.2, shuffle=True, random_state = 1724)

In [106]:
x_train.shape

(964, 162)

In [111]:
model = Sequential()
model.add(Dense(512, input_shape=(162,), activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(512,activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(256,activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(256,activation='swish'))
model.add(Dropout(0.3))
model.add(Dense(2))

# model compile
model.compile(loss='mse', optimizer='adam') 
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_21 (Dense)            (None, 512)               83456     
                                                                 
 dropout_13 (Dropout)        (None, 512)               0         
                                                                 
 dense_22 (Dense)            (None, 512)               262656    
                                                                 
 dropout_14 (Dropout)        (None, 512)               0         
                                                                 
 dense_23 (Dense)            (None, 256)               131328    
                                                                 
 dropout_15 (Dropout)        (None, 256)               0         
                                                                 
 dense_24 (Dense)            (None, 256)              

In [112]:
# model setting
early_stopping = keras.callbacks.EarlyStopping(patience = 20, restore_best_weights = True)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
history = model.fit(x_train, y_train, batch_size=256, epochs=3000, callbacks=[early_stopping, reduce_lr], validation_data=(x_valid, y_valid))

Epoch 1/3000
4/4 [==============================] - 1s 79ms/step - loss: 0.1617 - val_loss: 0.0436 - lr: 0.0010
Epoch 2/3000
4/4 [==============================] - 0s 18ms/step - loss: 0.0794 - val_loss: 0.0310 - lr: 0.0010
Epoch 3/3000
4/4 [==============================] - 0s 23ms/step - loss: 0.0440 - val_loss: 0.0363 - lr: 0.0010
Epoch 4/3000
4/4 [==============================] - 0s 22ms/step - loss: 0.0360 - val_loss: 0.0345 - lr: 0.0010
Epoch 5/3000
4/4 [==============================] - 0s 23ms/step - loss: 0.0374 - val_loss: 0.0221 - lr: 0.0010
Epoch 6/3000
4/4 [==============================] - 0s 18ms/step - loss: 0.0310 - val_loss: 0.0223 - lr: 0.0010
Epoch 7/3000
4/4 [==============================] - 0s 17ms/step - loss: 0.0269 - val_loss: 0.0213 - lr: 0.0010
Epoch 8/3000
4/4 [==============================] - 0s 19ms/step - loss: 0.0258 - val_loss: 0.0175 - lr: 0.0010
Epoch 9/3000
4/4 [==============================] - 0s 18ms/step - loss: 0.0225 - val_loss: 0.0153 - lr:

In [113]:
model.save('nn_model.h5')
# model = keras.models.load_model('nn_model.h5')

In [114]:
#train 예측 
train_predictions = model.predict(x_train)

#train 역변환 
val = target_scaler.inverse_transform(y_train)
predictions = target_scaler.inverse_transform(train_predictions)

#train RMSE
train_RMSE = mean_squared_error(val, predictions)**0.5

#test 예측 
valid_predictions = model.predict(x_valid)

#test 역변환 
val = target_scaler.inverse_transform(y_valid)
predictions = target_scaler.inverse_transform(valid_predictions)

#test RMSE
valid_RMSE = mean_squared_error(val, predictions)**0.5

print('train_RMSE:', train_RMSE)
print('valid_RMSE:', valid_RMSE)

train_RMSE: 86.11722603279989
valid_RMSE: 93.82694011358019


성능평가 마쳤으면 그냥 원래 test셋으로 submisson 준비해야지

In [115]:
test_scaled = data_scaler.transform(test_data)

test_predictions = model.predict(test_scaled)

test_predictions.shape

(50, 2)

In [116]:
y_pred = target_scaler.inverse_transform(test_predictions)
y_pred

array([[ 997.8457 ,  363.30164],
       [ 941.9089 ,  259.8381 ],
       [ 697.846  ,  351.3985 ],
       [1238.5563 ,  523.2933 ],
       [1013.96045,  415.46964],
       [ 986.3201 ,  198.82721],
       [ 957.58185,  474.6879 ],
       [ 704.96387,  407.31628],
       [1274.4165 ,  518.8286 ],
       [1036.8262 ,  428.41617],
       [ 856.45575,  261.60178],
       [1303.0995 ,  559.1429 ],
       [1085.7218 ,  543.382  ],
       [1040.9995 ,  378.65015],
       [ 955.1886 ,  436.33945],
       [ 701.5822 ,  316.11996],
       [1265.5619 ,  594.68274],
       [1095.9153 ,  557.9162 ],
       [ 966.8305 ,  454.02753],
       [ 924.1341 ,  498.54605],
       [ 661.45966,  329.5539 ],
       [1075.357  ,  559.8971 ],
       [1009.05316,  392.59277],
       [1005.9133 ,  538.8943 ],
       [ 693.0193 ,  404.5811 ],
       [1316.551  ,  605.12524],
       [1146.6992 ,  605.7111 ],
       [ 947.9522 ,  349.85864],
       [ 947.8426 ,  289.04092],
       [ 740.59155,  366.17328],
       [12

In [117]:
y_pred

array([[ 997.8457 ,  363.30164],
       [ 941.9089 ,  259.8381 ],
       [ 697.846  ,  351.3985 ],
       [1238.5563 ,  523.2933 ],
       [1013.96045,  415.46964],
       [ 986.3201 ,  198.82721],
       [ 957.58185,  474.6879 ],
       [ 704.96387,  407.31628],
       [1274.4165 ,  518.8286 ],
       [1036.8262 ,  428.41617],
       [ 856.45575,  261.60178],
       [1303.0995 ,  559.1429 ],
       [1085.7218 ,  543.382  ],
       [1040.9995 ,  378.65015],
       [ 955.1886 ,  436.33945],
       [ 701.5822 ,  316.11996],
       [1265.5619 ,  594.68274],
       [1095.9153 ,  557.9162 ],
       [ 966.8305 ,  454.02753],
       [ 924.1341 ,  498.54605],
       [ 661.45966,  329.5539 ],
       [1075.357  ,  559.8971 ],
       [1009.05316,  392.59277],
       [1005.9133 ,  538.8943 ],
       [ 693.0193 ,  404.5811 ],
       [1316.551  ,  605.12524],
       [1146.6992 ,  605.7111 ],
       [ 947.9522 ,  349.85864],
       [ 947.8426 ,  289.04092],
       [ 740.59155,  366.17328],
       [12

In [118]:
submission = pd.read_csv('sample_submission.csv')

In [119]:
submission['중식계'] = y_pred[:,0]
submission['석식계'] = y_pred[:,1]

In [120]:
submission

,일자,중식계,석식계
0,2021-01-27,997.845703,363.301636
1,2021-01-28,941.908875,259.838104
2,2021-01-29,697.846008,351.398499
3,2021-02-01,1238.556274,523.293274
4,2021-02-02,1013.960449,415.469635
5,2021-02-03,986.320129,198.827209
6,2021-02-04,957.581848,474.687897
7,2021-02-05,704.963867,407.316284
8,2021-02-08,1274.416504,518.828613
9,2021-02-09,1036.826172,428.416168


식수 계는 소수점이 없으므로 이것도 제출해본다.

In [121]:
submission1 = submission.round(0)

In [122]:
submission1

,일자,중식계,석식계
0,2021-01-27,998.0,363.0
1,2021-01-28,942.0,260.0
2,2021-01-29,698.0,351.0
3,2021-02-01,1239.0,523.0
4,2021-02-02,1014.0,415.0
5,2021-02-03,986.0,199.0
6,2021-02-04,958.0,475.0
7,2021-02-05,705.0,407.0
8,2021-02-08,1274.0,519.0
9,2021-02-09,1037.0,428.0


In [123]:
submission.to_csv('submission_NNmodel.csv', index = False)
submission1.to_csv('submission_NNmodel_round.csv', index = False)